In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

def create_weekly_features(data):
    """Create weekly aggregated features for stock price prediction."""
    df_weekly = data.copy()
    df_weekly['Week'] = df_weekly.index.isocalendar().week
    df_weekly['Month'] = df_weekly.index.month
    df_weekly['Year'] = df_weekly.index.year

    # Tính toán các đặc trưng theo tuần
    df_weekly = df_weekly.groupby(['Year', 'Week']).agg({
        'Adj Close': ['mean', 'std', 'min', 'max', 'last'],
        'Week': 'first',
        'Month': 'first'
    }).reset_index()

    df_weekly.columns = ['Year', 'Week', 'weekly_mean', 'weekly_std', 'weekly_min',
                        'weekly_max', 'weekly_last', 'Week_num', 'Month']

    # Tạo lag features - tăng lên 28 lags
    for i in range(1, 29):  # Tăng số lượng lag features lên 28
        df_weekly[f'weekly_mean_lag_{i}'] = df_weekly['weekly_mean'].shift(i)
        df_weekly[f'weekly_std_lag_{i}'] = df_weekly['weekly_std'].shift(i)

    # Tạo rolling features
    windows = [2, 3, 4, 5, 7, 14, 21, 28]  # Thêm các windows mới
    for window in windows:
        df_weekly[f'rolling_mean_{window}'] = df_weekly['weekly_mean'].shift(1).rolling(window=window).mean()
        df_weekly[f'rolling_std_{window}'] = df_weekly['weekly_mean'].shift(1).rolling(window=window).std()

    # Tạo rate of change features
    for i in range(1, 29):  # Tăng số lượng ROC features lên 28
        df_weekly[f'roc_{i}'] = df_weekly['weekly_mean'].pct_change(periods=i) * 100

    df_weekly['Month_sin'] = np.sin(2 * np.pi * df_weekly['Month']/12)
    df_weekly['Month_cos'] = np.cos(2 * np.pi * df_weekly['Month']/12)

    return df_weekly.dropna()

def calculate_errors(y_true, y_pred):
    """Tính toán sai số."""
    return y_true - y_pred

def create_error_features(errors):
    """Tạo đặc trưng từ sai số."""
    error_features = pd.DataFrame()

    # Tạo lag features cho sai số
    for i in range(1, 4):  # Giảm số lượng lag
        error_features[f'error_lag_{i}'] = pd.Series(errors).shift(i)

    # Tạo rolling features cho sai số
    error_features['error_ma_3'] = pd.Series(errors).shift(1).rolling(window=3).mean()
    error_features['error_std_3'] = pd.Series(errors).shift(1).rolling(window=3).std()

    return error_features.fillna(0)

def update_features(current_data, new_prediction):
    """Cập nhật đặc trưng cho lần dự đoán tiếp theo."""
    updated_data = current_data.copy()

    # Cập nhật lag features
    for i in range(28, 1, -1):  # Cập nhật cho 28 lags
        updated_data[f'weekly_mean_lag_{i}'] = updated_data[f'weekly_mean_lag_{i-1}'].values[0]
    updated_data['weekly_mean_lag_1'] = float(new_prediction)

    # Cập nhật các features khác
    lag_values = [float(new_prediction)]
    for i in range(1, 28):  # Cập nhật cho 28 lags
        lag_values.append(float(updated_data[f'weekly_mean_lag_{i}'].values[0]))

    # Cập nhật rolling features
    windows = [2, 3, 4, 5, 7, 14, 21, 28]  # Cập nhật windows
    for window in windows:
        if len(lag_values) >= window:
            updated_data[f'rolling_mean_{window}'] = float(np.mean(lag_values[:window]))
            updated_data[f'rolling_std_{window}'] = float(np.std(lag_values[:window]))

    # Cập nhật ROC
    for i in range(1, 29):  # Cập nhật cho 28 ROC
        if i < len(lag_values):
            prev_value = lag_values[i]
            if prev_value != 0:
                updated_data[f'roc_{i}'] = float(((new_prediction - prev_value) / prev_value) * 100)
            else:
                updated_data[f'roc_{i}'] = 0.0

    return updated_data

def predict_with_error_correction(price_model, error_model, scaler, pca, last_week_data,
                                last_errors_data, n_weeks=1):
    """Dự đoán với hiệu chỉnh sai số."""
    predictions = []
    error_corrections = []

    current_price_data = last_week_data.copy()
    current_error_data = last_errors_data.copy()

    for _ in range(n_weeks):
        # Dự đoán giá
        X_scaled = scaler.transform(current_price_data)
        X_pca = pca.transform(X_scaled)
        price_pred = float(price_model.predict(X_pca)[0])

        # Dự đoán sai số
        error_pred = float(error_model.predict(current_error_data.values.reshape(1, -1))[0])

        # Hiệu chỉnh dự đoán
        corrected_pred = price_pred + error_pred
        predictions.append(corrected_pred)
        error_corrections.append(error_pred)

        # Cập nhật dữ liệu
        current_price_data = update_features(current_price_data, corrected_pred)
        current_error_data = pd.DataFrame([error_pred] + list(current_error_data.iloc[0][:-1])).T
        current_error_data.columns = last_errors_data.columns

    return predictions, error_corrections

def calculate_metrics(y_true, y_pred):
    """Tính toán các chỉ số đánh giá."""
    mae = np.mean(np.abs(y_true - y_pred))
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # Tính Directional Accuracy
    y_true_direction = np.diff(y_true) > 0
    y_pred_direction = np.diff(y_pred) > 0
    da = np.mean(y_true_direction == y_pred_direction) * 100

    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'DA': da
    }

def main():
    # Load data
    file_path = 'Toyota_Data.csv'
    df = pd.read_csv(file_path)
    df = df.dropna()
    df = df.drop(['High', 'Low', 'Close', 'Open', 'Volume'], axis=1)
    df.set_index('Date', inplace=True)
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()

    # Split data
    df_train = df['2004/01/01':'2024/12/31']  # Gộp tất cả dữ liệu

    # Create features
    features_df = create_weekly_features(df_train)

    # Prepare data
    X = features_df.drop(['weekly_mean', 'Year', 'Week', 'Month'], axis=1)
    y = features_df['weekly_mean']
    print("Số lượng cột trong X:", len(X.columns))
    # Chia dữ liệu thành train và test
    test_size = 24  # 24 tuần cuối làm test
    X_train = X[:-test_size]
    X_test = X[-test_size:]
    y_train = y[:-test_size]
    y_test = y[-test_size:]

    # Preprocess
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    print(f"Number of components after PCA: {pca.n_components_}")

    # Train price prediction model
    price_model = XGBRegressor(
        n_estimators=2000,
        learning_rate=0.01,
        max_depth=5,
        random_state=42
    )
    price_model.fit(X_train_pca, y_train)

    # Model 1: Direct prediction on test set
    y_pred_test = price_model.predict(X_test_pca)

    # Model 2: Sequential prediction starting from train set
    sequential_predictions = []
    current_data = X_train.iloc[[-1]].copy()  # Bắt đầu từ tuần cuối của tập train

    for _ in range(test_size):
        print(X.iloc[-1])
        # Predict next week
        X_scaled = scaler.transform(current_data)
        X_pca = pca.transform(X_scaled)
        pred = float(price_model.predict(X_pca)[0])
        sequential_predictions.append(pred)

        # Update features for next prediction
        current_data = update_features(current_data, pred)

    # Calculate metrics for both models
    model1_metrics = calculate_metrics(y_test.values, y_pred_test)
    model2_metrics = calculate_metrics(y_test.values, sequential_predictions)

    print("\nMetrics for Model 1 (Direct Test Prediction):")
    print(f"MAE: {model1_metrics['MAE']:.2f}")
    print(f"RMSE: {model1_metrics['RMSE']:.2f}")
    print(f"DA: {model1_metrics['DA']:.2f}%")

    print("\nMetrics for Model 2 (Sequential Train Prediction):")
    print(f"MAE: {model2_metrics['MAE']:.2f}")
    print(f"RMSE: {model2_metrics['RMSE']:.2f}")
    print(f"DA: {model2_metrics['DA']:.2f}%")

    # Visualize
    plt.figure(figsize=(12, 6))

    # Plot data
    weeks = range(len(y_test))

    # Plot actual values
    plt.plot(weeks, y_test.values, label="Actual Values", color='blue', linewidth=2)

    # Plot Model 1 predictions
    plt.plot(weeks, y_pred_test, label="Model 1 (Test)", color='red', linestyle='--')

    # Plot Model 2 predictions
    plt.plot(weeks, sequential_predictions, label="Model 2 (Train)", color='green', linestyle='--')

    plt.legend()
    plt.grid(True)
    plt.title("Comparison of Two Prediction Models")
    plt.xlabel("Week")
    plt.ylabel("Price")

    # Add metrics to plot
    metrics_text = (
        f'Model 1 (Test):\n'
        f'MAE: {model1_metrics["MAE"]:.2f}\n'
        f'RMSE: {model1_metrics["RMSE"]:.2f}\n'
        f'DA: {model1_metrics["DA"]:.2f}%\n\n'
        f'Model 2 (Train):\n'
        f'MAE: {model2_metrics["MAE"]:.2f}\n'
        f'RMSE: {model2_metrics["RMSE"]:.2f}\n'
        f'DA: {model2_metrics["DA"]:.2f}%'
    )

    plt.text(0.02, 0.98, metrics_text,
             transform=plt.gca().transAxes,
             bbox=dict(facecolor='white', alpha=0.8),
             verticalalignment='top')

    plt.tight_layout()
    plt.show()

    # Plot errors for both models
    plt.figure(figsize=(15, 6))

    # Model 1 errors
    model1_errors = y_test.values - y_pred_test
    plt.plot(weeks, model1_errors, label='Model 1 Errors', color='red')

    # Model 2 errors
    model2_errors = y_test.values - sequential_predictions
    plt.plot(weeks, model2_errors, label='Model 2 Errors', color='green')

    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.grid(True)
    plt.legend()
    plt.title('Prediction Errors for Both Models')
    plt.xlabel('Week')
    plt.ylabel('Error (Actual - Predicted)')
    plt.show()
    # Plot feature importance for original features
    plt.figure(figsize=(15, 10))

    # Get original feature names
    original_feature_names = X.columns.tolist()

    # Create a new model to get importance of original features
    original_model = XGBRegressor(
        n_estimators=2000,
        learning_rate=0.01,
        max_depth=5,
        random_state=42
    )
    original_model.fit(X_train, y_train)

    # Get feature importance
    original_importance = original_model.feature_importances_

    # Sort features by importance
    sorted_idx = np.argsort(original_importance)

    # Create a DataFrame for better visualization
    feature_importance_df = pd.DataFrame({
        'Feature': [original_feature_names[i] for i in sorted_idx],
        'Importance': original_importance[sorted_idx]
    })

    # Plot feature importance
    plt.figure(figsize=(15, 10))
    plt.barh(range(len(sorted_idx)), original_importance[sorted_idx])
    plt.yticks(range(len(sorted_idx)), [original_feature_names[i] for i in sorted_idx])
    plt.title('Feature Importance for Stock Price Prediction')
    plt.xlabel('Feature Importance Score')
    plt.ylabel('Features')

    # Add importance values on the bars
    for i, v in enumerate(original_importance[sorted_idx]):
        plt.text(v, i, f' {v:.4f}', va='center')

    plt.tight_layout()
    plt.show()

    # Print feature importance in a table format
    print("\nFeature Importance Table:")
    print("=" * 80)
    print(f"{'Feature':<40} {'Importance':<20}")
    print("-" * 80)
    for feature, importance in zip(original_feature_names, original_importance):
        print(f"{feature:<40} {importance:.6f}")
    print("=" * 80)

    # Group features by type and calculate average importance
    feature_groups = {
        'Lag Features': [f for f in original_feature_names if 'lag' in f],
        'Rolling Features': [f for f in original_feature_names if 'rolling' in f],
        'Rate of Change': [f for f in original_feature_names if 'roc' in f],
        'Seasonal Features': [f for f in original_feature_names if 'Month' in f]
    }

    # Calculate average importance for each group
    group_importance = {}
    for group_name, features in feature_groups.items():
        group_importance[group_name] = np.mean([original_importance[original_feature_names.index(f)] for f in features])

    # Plot group importance
    plt.figure(figsize=(12, 6))
    plt.barh(range(len(group_importance)), list(group_importance.values()))
    plt.yticks(range(len(group_importance)), list(group_importance.keys()))
    plt.title('Average Feature Importance by Group')
    plt.xlabel('Average Importance Score')
    plt.ylabel('Feature Groups')

    # Add importance values on the bars
    for i, v in enumerate(group_importance.values()):
        plt.text(v, i, f' {v:.4f}', va='center')

    plt.tight_layout()
    plt.show()

    # Print group importance in a table format
    print("\nGroup Feature Importance:")
    print("=" * 50)
    print(f"{'Group':<20} {'Average Importance':<20}")
    print("-" * 50)
    for group, importance in group_importance.items():
        print(f"{group:<20} {importance:.6f}")
    print("=" * 50)

if __name__ == "__main__":
    main()